# Influence of setting `ibrav`

In [1]:
from aiida import orm, load_profile
from aiida_quantumespresso.workflows.pw.base import PwBaseWorkChain
from pathlib import Path
from aiida.engine import run_get_node

from ase import io

load_profile()

Profile<uuid='8f0080521aaf4d0cbdb91e33773008fd' name='mbx'>

In [2]:
atoms = io.read(Path('scf-ibrav2.in'))
structure = orm.StructureData(ase=atoms)
code = orm.load_code('qe-7.2-pw@localhost')

## Run without `ibrav`

In [3]:
builder = PwBaseWorkChain.get_builder_from_protocol(code=code, structure=structure)

results, node = run_get_node(builder)

04/23/2023 02:35:49 PM <12893> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [782|PwBaseWorkChain|run_process]: launching PwCalculation<787> iteration #1
04/23/2023 02:35:51 PM <12893> aiida.scheduler.direct: [WARNING] Unrecognized job_state '?' for job id 13004
04/23/2023 02:35:51 PM <12893> aiida.scheduler.direct: [WARNING] Unrecognized job_state '?' for job id 13005
04/23/2023 02:35:51 PM <12893> aiida.scheduler.direct: [WARNING] Unrecognized job_state '?' for job id 13006
04/23/2023 02:36:01 PM <12893> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [782|PwBaseWorkChain|results]: work chain completed after 1 iterations
04/23/2023 02:36:01 PM <12893> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [782|PwBaseWorkChain|on_terminated]: remote folders will not be cleaned


In [4]:
results['output_parameters'].get_dict()['wall_time']

'     10.56s '

In [5]:
node.called[-1].base.repository.list_object_names()
print(node.called[-1].base.repository.get_object_content('aiida.in'))

&CONTROL
  calculation = 'scf'
  etot_conv_thr =   2.0000000000d-05
  forc_conv_thr =   1.0000000000d-04
  max_seconds =   4.1040000000d+04
  outdir = './out/'
  prefix = 'aiida'
  pseudo_dir = './pseudo/'
  tprnfor = .true.
  tstress = .true.
  verbosity = 'high'
/
&SYSTEM
  degauss =   1.0000000000d-02
  ecutrho =   4.0000000000d+02
  ecutwfc =   5.0000000000d+01
  ibrav = 0
  nat = 2
  nosym = .false.
  ntyp = 2
  occupations = 'smearing'
  smearing = 'cold'
/
&ELECTRONS
  conv_thr =   4.0000000000d-10
  electron_maxstep = 80
  mixing_beta =   4.0000000000d-01
/
ATOMIC_SPECIES
Mg     24.305 Mg.pbesol-n-kjpaw_psl.0.3.0.UPF
O      15.999 O.pbesol-n-kjpaw_psl.0.1.UPF
ATOMIC_POSITIONS angstrom
Mg           0.0000000000       0.0000000000       0.0000000000
O            2.1266191896       2.1266191896       2.1266191896
K_POINTS automatic
18 18 18 0 0 0
CELL_PARAMETERS angstrom
     -2.1266191896       0.0000000000       2.1266191896
      0.0000000000       2.1266191896       2.12661918

## Run with `ibrav`

In [6]:
builder = PwBaseWorkChain.get_builder_from_protocol(code=code, structure=structure)

builder.pw.parameters['SYSTEM']['ibrav'] = 2

# Note that this is not possible - Also see https://github.com/aiidateam/aiida-quantumespresso/issues/922 
# builder.pw.parameters['SYSTEM']['celldm(1)'] = 8.0374557182  

results, node = run_get_node(builder)

04/23/2023 02:36:02 PM <12893> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [798|PwBaseWorkChain|run_process]: launching PwCalculation<803> iteration #1
04/23/2023 02:36:14 PM <12893> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [798|PwBaseWorkChain|results]: work chain completed after 1 iterations
04/23/2023 02:36:14 PM <12893> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [798|PwBaseWorkChain|on_terminated]: remote folders will not be cleaned


In [7]:
results['output_parameters'].get_dict()['wall_time']

'     10.61s '

In [8]:
print(node.called[-1].base.repository.get_object_content('aiida.in'))

&CONTROL
  calculation = 'scf'
  etot_conv_thr =   2.0000000000d-05
  forc_conv_thr =   1.0000000000d-04
  max_seconds =   4.1040000000d+04
  outdir = './out/'
  prefix = 'aiida'
  pseudo_dir = './pseudo/'
  tprnfor = .true.
  tstress = .true.
  verbosity = 'high'
/
&SYSTEM
  a =   4.2532383792d+00
  degauss =   1.0000000000d-02
  ecutrho =   4.0000000000d+02
  ecutwfc =   5.0000000000d+01
  ibrav = 2
  nat = 2
  nosym = .false.
  ntyp = 2
  occupations = 'smearing'
  smearing = 'cold'
/
&ELECTRONS
  conv_thr =   4.0000000000d-10
  electron_maxstep = 80
  mixing_beta =   4.0000000000d-01
/
ATOMIC_SPECIES
Mg     24.305 Mg.pbesol-n-kjpaw_psl.0.3.0.UPF
O      15.999 O.pbesol-n-kjpaw_psl.0.1.UPF
ATOMIC_POSITIONS angstrom
Mg           0.0000000000       0.0000000000       0.0000000000
O            2.1266191896       2.1266191896       2.1266191896
K_POINTS automatic
18 18 18 0 0 0



## `parent_folder` creator

In [9]:
# First run
builder = PwBaseWorkChain.get_builder_from_protocol(code=code, structure=structure)

results, node = run_get_node(builder)

# Restart
restart_builder = PwBaseWorkChain.get_builder_from_protocol(code=code, structure=structure)

parameters = restart_builder.pw.parameters.get_dict()
parameters['ELECTRONS']['startingpot'] = 'file'
restart_builder.pw.parameters = orm.Dict(parameters)
restart_builder.pw.parent_folder = node.outputs.remote_folder

restart_results, restart_node = run_get_node(restart_builder)

restart_node.inputs.pw.parent_folder.creator

04/23/2023 02:36:15 PM <12893> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [814|PwBaseWorkChain|run_process]: launching PwCalculation<819> iteration #1
04/23/2023 02:36:26 PM <12893> aiida.scheduler.direct: [WARNING] Unrecognized job_state '?' for job id 13758
04/23/2023 02:36:27 PM <12893> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [814|PwBaseWorkChain|results]: work chain completed after 1 iterations
04/23/2023 02:36:27 PM <12893> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [814|PwBaseWorkChain|on_terminated]: remote folders will not be cleaned
04/23/2023 02:36:28 PM <12893> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [830|PwBaseWorkChain|run_process]: launching PwCalculation<835> iteration #1
04/23/2023 02:36:37 PM <12893> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [830|PwBaseWorkChain|results]: work chain completed after 1 iterations
04/23/2023 02:36:37 PM <12893> ai

<CalcJobNode: uuid: ed2e8455-500e-42a5-a4e3-7761f255a25d (pk: 819) (aiida.calculations:quantumespresso.pw)>

You can also do the following, but only because the `Dict` node is not stored!

In [10]:
parent_folder = node.outputs.remote_folder

restart_builder = PwBaseWorkChain.get_builder_from_protocol(code=code, structure=structure)

restart_builder.pw.parameters['ELECTRONS']['startingpot'] = 'file'
restart_builder.pw.parent_folder = parent_folder

test_restults, test_node = run_get_node(restart_builder)

04/23/2023 02:36:38 PM <12893> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [846|PwBaseWorkChain|run_process]: launching PwCalculation<851> iteration #1
04/23/2023 02:36:48 PM <12893> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [846|PwBaseWorkChain|results]: work chain completed after 1 iterations
04/23/2023 02:36:48 PM <12893> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [846|PwBaseWorkChain|on_terminated]: remote folders will not be cleaned
